In [9]:
#FINAL FINAL FINAL

import cvxpy as cp
import numpy as np

# Data setup
num_students = 6
num_universities = 3

student_data = np.array([[3.9, 1500, 1],  # Student 1 - rank 3
                         [4.0, 1600, 1],  # Student 2 - rank 1
                         [4.0, 1600, 1],  # Student 3 - rank 2
                         [3.6, 1350, 5],  # Student 4 - rank 5 
                         [3.7, 1450, 1],  # Student 5 - rank 4 
                         [3.9, 1550, 1]])  # Student 6 - rank 6

university_requirements = np.array([[3.0, 1200, 20],  # University 1
                                    [3.5, 1300, 20],  # University 2
                                    [3.2, 1250, 20]]) # University 3


preference_scores = np.array([[3, 2, 1], 
                               [3, 2, 1],
                               [3, 2, 1],
                               [3, 2, 1],
                               [3, 2, 1],
                               [3, 2, 1]])

university_capacities = np.array([10, 10, 10])

x = cp.Variable((num_students, num_universities), boolean=True)

# Sort students
sorted_indices = np.lexsort((student_data[:, 2], student_data[:, 1], -student_data[:, 0]))
sorted_student_data = student_data[sorted_indices]
sorted_preference_scores = preference_scores[sorted_indices]

objective = cp.Maximize(cp.sum(cp.multiply(sorted_preference_scores, x)))

constraints = []

#Eligibility constraint
for i in range(num_students):
    for j in range(num_universities):
        gpa, sat, rank = sorted_student_data[i]
        min_gpa, min_sat, max_rank = university_requirements[j]
        if gpa >= min_gpa and sat >= min_sat and rank <= max_rank:
            eligibility_matrix[i, j] = 1  
            constraints.append(x[i, j] == 1)  
        else:
            constraints.append(x[i, j] == 0)  

# Capacity Constraint
for j in range(num_universities):
    constraints.append(cp.sum(x[:, j]) <= university_capacities[j])

problem = cp.Problem(objective, constraints)
problem.solve()

print("Total preference score:", objective.value)
print("Matching results:")
for i, original_index in enumerate(sorted_indices):
    admitted_universities = []
    for j in range(num_universities):
        if x.value is not None and x.value[i, j] > 0.5:
            admitted_universities.append(j + 1)
    if admitted_universities:
        print(f"Student {original_index + 1} admitted to University/Universities: {', '.join(map(str, admitted_universities))}")
    else:
        print(f"Student {original_index + 1} not admitted to any university")

Total preference score: 36.0
Matching results:
Student 2 admitted to University/Universities: 1, 2, 3
Student 3 admitted to University/Universities: 1, 2, 3
Student 1 admitted to University/Universities: 1, 2, 3
Student 6 admitted to University/Universities: 1, 2, 3
Student 5 admitted to University/Universities: 1, 2, 3
Student 4 admitted to University/Universities: 1, 2, 3


In [2]:
# NEW FINAL
import cvxpy as cp
import numpy as np
import pandas as pd

st_df = pd.read_csv("StudentAdmissionData2.csv")
uni_df = pd.read_csv("Top20UsUnis.csv")
pred_df = pd.read_csv("StudentPrefData.csv")

uni_df2 = uni_df.drop("college", axis=1)

student_data = np.array(st_df)
university_requirements = np.array(uni_df2)
preference_scores = np.array(pred_df)

# Data setup
num_students = 1000
num_universities = 20

university_capacities = np.full(20, 5)
x = cp.Variable((num_students, 20), boolean=True)

# Sort students
sorted_indices = np.lexsort((-student_data[:, 2], -student_data[:, 1], -student_data[:, 0]))
sorted_student_data = student_data[sorted_indices]
sorted_preference_scores = preference_scores[sorted_indices]

objective = cp.Maximize(cp.sum(cp.multiply(sorted_preference_scores, x)))

constraints = []
eligibility_matrix = np.zeros((num_students, 20), dtype=int)

#Eligibility constraint
for i in range(num_students):
    for j in range(num_universities):
        gpa, sat, rank = sorted_student_data[i]
        min_gpa, min_sat, max_rank = university_requirements[j]
        if gpa >= min_gpa and sat >= min_sat and rank <= max_rank and sorted_preference_scores[i, j] > 0:
            eligibility_matrix[i, j] = 1  
            constraints.append(x[i, j] <= 1)  
        else:
            constraints.append(x[i, j] <= 0)  

##### Something is going wrong with the capacity constraint.

# Capacity Constraint - This should take the top students for each school, but if there are more students eligible then the capacity
#limit then no students are let in.

# old - goes over capacity constraint then no one is accepted
#for i in range(num_universities):
#    for j in range(num_students):
#        constraints.append(cp.sum(x[j, i]) <= university_capacities[i])


#for j in range(num_universities):
#    constraints.append(cp.sum(x[:, j]) <= university_capacities[j])

# Capacity Constraint
#for j in range(num_universities):
#    constraints.append(cp.sum(x[:, j]) <= university_capacities[j])

# Capacity constraint for universities
for j in range(num_universities):
    constraints.append(cp.sum(x[:, j]) <= university_capacities[j])  # University capacity

# Each student can only attend one university
for i in range(num_students):
    constraints.append(cp.sum(x[i, :]) <= 1)  # One university per student

# new - doesnt enforce capacity constraint
#for i in range(num_students):
#    for j in range(num_universities):
        #constraints.append(x[i, j] <= eligibility_matrix[i, j]) 
#        contraints.append(x[: 

problem = cp.Problem(objective, constraints)
problem.solve()

print("Total preference score:", objective.value)
print("Matching results:")
for i, original_index in enumerate(sorted_indices):
    admitted_universities = []
    for j in range(num_universities):
        if x.value is not None and x.value[i, j] > 0.5:
            admitted_universities.append(j + 1)
    if admitted_universities:
        print(f"Student {original_index + 1} admitted to University/Universities: {', '.join(map(str, admitted_universities))}")
    else:
        print(f"Student {original_index + 1} not admitted to any university")

Total preference score: 1619.0
Matching results:
Student 11 admitted to University/Universities: 18
Student 23 admitted to University/Universities: 19
Student 80 admitted to University/Universities: 17
Student 93 admitted to University/Universities: 7
Student 100 admitted to University/Universities: 15
Student 101 admitted to University/Universities: 9
Student 105 admitted to University/Universities: 3
Student 123 admitted to University/Universities: 11
Student 128 admitted to University/Universities: 8
Student 152 admitted to University/Universities: 14
Student 172 admitted to University/Universities: 10
Student 186 admitted to University/Universities: 9
Student 210 admitted to University/Universities: 2
Student 211 admitted to University/Universities: 4
Student 221 admitted to University/Universities: 8
Student 242 admitted to University/Universities: 13
Student 243 admitted to University/Universities: 7
Student 338 admitted to University/Universities: 11
Student 417 admitted to Univ

In [15]:
import cvxpy as cp
import numpy as np
import pandas as pd

st_df = pd.read_csv("StudentAdmissionData2.csv")
uni_df = pd.read_csv("Top20UsUnis.csv")
pred_df = pd.read_csv("StudentPrefData.csv")



uni_df2 = uni_df.drop("college", axis=1)

student_data = np.array(st_df)
university_requirements = np.array(uni_df2)
preference_scores = np.array(pred_df)

# Data setup
num_students = 1000
num_universities = 20

university_capacities = np.full(20, 5)
x = cp.Variable((num_students, 20), boolean=True)

# Sort students
sorted_indices = np.lexsort((-student_data[:, 2], -student_data[:, 1], -student_data[:, 0]))
sorted_student_data = student_data[sorted_indices]
sorted_preference_scores = preference_scores[sorted_indices]

objective = cp.Maximize(cp.sum(cp.multiply(sorted_preference_scores, x)))

constraints = []
eligibility_matrix = np.zeros((num_students, 20), dtype=int)

#Eligibility constraint
for i in range(num_students):
    for j in range(num_universities):
        gpa, sat, rank = sorted_student_data[i]
        min_gpa, min_sat, max_rank = university_requirements[j]
        if gpa >= min_gpa and sat >= min_sat and rank <= max_rank:
            eligibility_matrix[i, j] = 1  
            constraints.append(x[i, j] == 1)  
        else:
            constraints.append(x[i, j] == 0)  

##### Something is going wrong with the capacity constraint.

# Capacity Constraint - This should take the top students for each school, but if there are more students eligible then the capacity
#limit then no students are let in.

# old - goes over capacity constraint then no one is accepted
#for j in range(num_universities):
#    constraints.append(cp.sum(x[:, j]) <= university_capacities[j])

# new - doesnt enforce capacity constraint
for i in range(num_students):
    for j in range(num_universities):
        constraints.append(x[i, j] <= eligibility_matrix[i, j]) 

problem = cp.Problem(objective, constraints)
problem.solve()

print("Total preference score:", objective.value)
print("Matching results:")
for i, original_index in enumerate(sorted_indices):
    admitted_universities = []
    for j in range(num_universities):
        if x.value is not None and x.value[i, j] > 0.5:
            admitted_universities.append(j + 1)
    if admitted_universities:
        pd.set_option('display.max_rows', None)
        print(f"Student {original_index + 1} admitted to University/Universities: {', '.join(map(str, admitted_universities))}")
    else:
        pd.set_option('display.max_rows', None)
        print(f"Student {original_index + 1} not admitted to any university")

Total preference score: 358.0
Matching results:
Student 27 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Student 66 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Student 98 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Student 197 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Student 232 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Student 236 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Student 390 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Student 398 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
Stu

In [9]:
import cvxpy as cp
import numpy as np
import pandas as pd

st_df = pd.read_csv("StudentAdmissionData1.csv")
uni_df = pd.read_csv("Top20UsUnis.csv")
pred_df = pd.read_csv("StudentPrefData.csv")
st_df.drop("Student_ID", axis=1, inplace=True)


uni_df2 = uni_df.drop("college", axis=1)

student_data = np.array(st_df)
university_requirements = np.array(uni_df2)
preference_scores = np.array(pred_df)

# Data setup
num_students = 1000
num_universities = 20

university_capacities = np.full(20, 10)

x = cp.Variable((num_students, num_universities), boolean=True)

# Sort students
sorted_indices = np.lexsort((student_data[:, 2], student_data[:, 1], -student_data[:, 0]))
sorted_student_data = student_data[sorted_indices]
sorted_preference_scores = preference_scores[sorted_indices]

objective = cp.Maximize(cp.sum(cp.multiply(sorted_preference_scores, x)))

constraints = []
eligibility_matrix = np.zeros((num_students, num_universities), dtype=int)

#Eligibility constraint
for i in range(num_students):
    for j in range(num_universities):
        gpa, sat, rank = sorted_student_data[i]
        min_gpa, min_sat, max_rank = university_requirements[j]
        if gpa >= min_gpa and sat >= min_sat and rank <= max_rank:
            eligibility_matrix[i, j] = 1  
            constraints.append(x[i, j] == 1)  
        else:
            constraints.append(x[i, j] == 0)  

# Capacity Constraint
for j in range(num_universities):
    constraints.append(cp.sum(x[:, j]) <= university_capacities[j])

problem = cp.Problem(objective, constraints)
problem.solve()

print("Total preference score:", objective.value)
print("Matching results:")
for i, original_index in enumerate(sorted_indices):
    admitted_universities = []
    for j in range(num_universities):
        if x.value is not None and x.value[i, j] > 0.5:
            admitted_universities.append(j + 1)
    if admitted_universities:
    
        print(f"Student {original_index + 1} admitted to University/Universities: {', '.join(map(str, admitted_universities))}")
    #else:
    #    print(f"Student {original_index + 1} not admitted to any university")

Total preference score: 0.0
Matching results:


In [8]:
print(eligibility_matrix)
print()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
